In [ ]:
import torch
import wandb
from trl import DPOTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset


def dpo_format(example):
    """Format Anthropic HH data as a dictionary with 'prompt', 'chosen', and 'rejected' keys."""

    chosen_list = example['chosen'].split()
    rejected_list = example['rejected'].split()
    response_index = len(chosen_list) - 1 - chosen_list[::-1].index('Assistant:')

    prompt = ' '.join(chosen_list[:response_index])
    chosen = ' '.join(chosen_list[response_index:])
    rejected = ' '.join(rejected_list[response_index:])

    return {
        'prompt':prompt,
        'chosen':chosen,
        'rejected':rejected
    }


model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
training_data_path = './data/harmless-base/train.jsonl'
new_model = f"./models/{model_name}_dpo_no_metadata"

with open('./keys/wandb_key.txt', 'r') as file:
    wandb_key = file.read().strip()
wandb.login(key=wandb_key)

In [ ]:
# Load dataset
dataset = load_dataset('json', data_files={'train': training_data_path})['train'].map(dpo_format)
dataset

In [ ]:
# Load tokenizer and models

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

In [ ]:
# Set training configurations

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
    remove_unused_columns=False
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

In [ ]:
# Train!
dpo_trainer.train()